# Day 8: Space Image Format

[Aoc Site](https://adventofcode.com/2019/day/8)

## Part I

<p><span title="I'm not sorry.">The Elves' spirits are lifted when they realize you have an opportunity to reboot one of their Mars rovers, and so they are curious if you would spend a brief sojourn on Mars.</span> You land your ship near the rover.</p>
<p>When you reach the rover, you discover that it's already in the process of rebooting! It's just waiting for someone to enter a <a href="https://en.wikipedia.org/wiki/BIOS">BIOS</a> password. The Elf responsible for the rover takes a picture of the password (your puzzle input) and sends it to you via the Digital Sending Network.</p>
<p>Unfortunately, images sent via the Digital Sending Network aren't encoded with any normal encoding; instead, they're encoded in a special Space Image Format.  None of the Elves seem to remember why this is the case. They send you the instructions to decode it.</p>
<p>Images are sent as a series of digits that each represent the color of a single pixel.  The digits fill each row of the image left-to-right, then move downward to the next row, filling rows top-to-bottom until every pixel of the image is filled.</p>
<p>Each image actually consists of a series of identically-sized <em>layers</em> that are filled in this way. So, the first digit corresponds to the top-left pixel of the first layer, the second digit corresponds to the pixel to the right of that on the same layer, and so on until the last digit, which corresponds to the bottom-right pixel of the last layer.</p>
<p>For example, given an image <code>3</code> pixels wide and <code>2</code> pixels tall, the image data <code>123456789012</code> corresponds to the following image layers:</p>
<pre><code>Layer 1: 123
         456

Layer 2: 789
         012
</code></pre>
<p>The image you received is <em><code>25</code> pixels wide and <code>6</code> pixels tall</em>.</p>
<p>To make sure the image wasn't corrupted during transmission, the Elves would like you to find the layer that contains the <em>fewest <code>0</code> digits</em>.  On that layer, what is <em>the number of <code>1</code> digits multiplied by the number of <code>2</code> digits?</em></p>

In [1]:
# Prep Notebook
import pandas as pd
import numpy as np

import disfunctions as func

In [2]:
def get_file_content(path):
    with open() as f:
        return f.read()

In [3]:
t1input = '123456789012'

In [4]:
def source_to_df(data, width=3, height=2):
    df = pd.DataFrame(([d for d in data]), columns=['digits'])
    chars = len(data)
    layer_len = width * height
    layers = chars // layer_len
    extras = chars % layer_len
    print(chars, layer_len, layers, extras)
    df['iter'] = df.index
    df['layer'] = df.iter.apply(lambda x: x // layer_len)
    df['xpos'] = df.iter.apply(lambda x: x % width)
    df['ypos'] = df.iter.apply(lambda x: (x % layer_len) // width)
    df.drop(columns=['iter'], inplace=True)
    if extras:
        df.layer.iloc[-extras:] = -1
    
    return df
    
    

In [5]:
testdf = source_to_df(t1input)
testdf.head(27)

12 6 2 0


,digits,layer,xpos,ypos
0,1,0,0,0
1,2,0,1,0
2,3,0,2,0
3,4,0,0,1
4,5,0,1,1
5,6,0,2,1
6,7,1,0,0
7,8,1,1,0
8,9,1,2,0
9,0,1,0,1


<p>To begin, <a href="https://adventofcode.com/2019/day/8/input" target="_blank">get your puzzle input</a>.</p>

In [6]:
source_file = '../DataFiles/2019_08.txt'
p1input = get_file_content(source_file)
len(p1input)

TypeError: open() missing required argument 'file' (pos 1)

In [ ]:
srcdf=source_to_df(p1input, width=25, height=6)
srcdf.describe()

In [ ]:
srcdf.layer.value_counts()

In [ ]:
srcxtras=srcdf[srcdf.layer==-1]
srcdf = srcdf[srcdf.layer!=-1]
srcdf.describe()

In [ ]:
chk_layer = srcdf[srcdf.digits == '0'].layer.value_counts().idxmin()

In [ ]:
chkdf = srcdf[srcdf.layer == chk_layer].digits
chkdf.value_counts()

In [ ]:
num1s = chkdf[chkdf=='1'].count()
num2s = chkdf[chkdf=='2'].count()

In [ ]:
ansp1 = num1s * num2s
ansp1

In [ ]:
actp1 = 2286
ansp1 == actp1

### That's the right answer! You are one gold star closer to rescuing Santa.

# Part II

<p>Now you're ready to decode the image. The image is rendered by stacking the layers and aligning the pixels with the same positions in each layer. The digits indicate the color of the corresponding pixel: <code>0</code> is black, <code>1</code> is white, and <code>2</code> is transparent.</p>
<p>The layers are rendered with the first layer in front and the last layer in back. So, if a given position has a transparent pixel in the first and second layers, a black pixel in the third layer, and a white pixel in the fourth layer, the final image would have a <em>black</em> pixel at that position.</p>
<p>For example, given an image <code>2</code> pixels wide and <code>2</code> pixels tall, the image data <code>0222112222120000</code> corresponds to the following image layers:</p>
<pre><code>Layer 1: <em>0</em>2
         22

Layer 2: 1<em>1</em>
         22

Layer 3: 22
         <em>1</em>2

Layer 4: 00
         0<em>0</em>
</code></pre>
<p>Then, the full image can be found by determining the top visible pixel in each position:</p>
<ul>
<li>The top-left pixel is <em>black</em> because the top layer is <code>0</code>.</li>
<li>The top-right pixel is <em>white</em> because the top layer is <code>2</code> (transparent), but the second layer is <code>1</code>.</li>
<li>The bottom-left pixel is <em>white</em> because the top two layers are <code>2</code>, but the third layer is <code>1</code>.</li>
<li>The bottom-right pixel is <em>black</em> because the only visible pixel in that position is <code>0</code> (from layer 4).</li>
</ul>
<p>So, the final image looks like this:</p>
<pre><code>01
10
</code></pre>
<p><em>What message is produced after decoding your image?</em></p>


In [ ]:
def drill_layers(df, xpos, ypos):
    chkdf = df[(df.xpos == xpos) & (df.ypos == ypos) & (df.digits != '2')].digits.head(1)
    chkval = chkdf.max()
    chkval = 'O' if chkval == '1' else ' '
    return chkval

In [ ]:
def interpret_source(df):
    xlim = df.xpos.max() + 1
    ylim = df.ypos.max() + 1
    yvals = []
    for y in range(ylim):
        xvals = []
        for x in range(xlim):
            xval=drill_layers(df, x, y)            
            xvals.append(xval)
        xrow = ''.join(xvals)
        yvals.append(xrow)
    return '\n'.join(yvals)

In [ ]:
drill_layers(srcdf, 0, 0)

In [ ]:
img = interpret_source(srcdf)
print(img)

In [ ]:
ansp2 = 'CJZLP'

In [ ]:
actp2 = 'CJZLP'
ansp2 == actp2

### That's the right answer! You are one gold star closer to rescuing Santa.